In [1]:
# Import Required Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import chirp
from scipy.fft import fft
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import librosa

# Set Constants
SR = 16000  # Sample Rate
DURATION = 2.5  # Signal Duration in Seconds

# File Paths
FEATURES_CSV = '/home/aditya_tilak/dataset/Aphidoletes_aphidimyza_features.csv'
AUDIO_FOLDER = '/mnt/external_disk/InsectSound1000'
INSECT_NAME = 'Aphidoletes_'

# Step 1: Load and Filter Dataset
df = pd.read_csv(FEATURES_CSV)


# Filter to relevant dominant frequencies
RELEVANT_FREQ_RANGE = (0, 200)  # Based on analysis
df_filtered = df[(df['dominant_freq'] >= RELEVANT_FREQ_RANGE[0]) & (df['dominant_freq'] <= RELEVANT_FREQ_RANGE[1])]

# Analyze Key Descriptive Statistics for Filtered Data
def analyze_dominant_freq(df):
    description = df['dominant_freq'].describe()
    print("Descriptive Statistics for Dominant Frequency:")
    print(description)
    return description

analyze_dominant_freq(df_filtered)




Descriptive Statistics for Dominant Frequency:
count    14044.000000
mean        20.782669
std         23.443180
min          0.000000
25%          0.000000
50%         21.533203
75%         32.299805
max        161.499023
Name: dominant_freq, dtype: float64


count    14044.000000
mean        20.782669
std         23.443180
min          0.000000
25%          0.000000
50%         21.533203
75%         32.299805
max        161.499023
Name: dominant_freq, dtype: float64

In [2]:
# Step 2: Create Natural Sound Profile
# Extract Features: Spectral Properties, Amplitude, Energy, etc.
def extract_sound_profile(df):
    profile = {
        "Mean Frequency": df['dominant_freq'].mean(),
        "Median Frequency": df['dominant_freq'].median(),
        "Mode Frequency": df['dominant_freq'].mode()[0],
        "Amplitude Range": (df['amplitude'].min(), df['amplitude'].max()),
        "Spectral Centroid Mean": df['spectral_centroid'].mean(),
        "Spectral Bandwidth Mean": df['spectral_bandwidth'].mean(),
        "RMS Energy Mean": df['rms_energy'].mean(),
        "Zero-Crossing Rate Mean": df['zero_crossing_rate'].mean(),
        "Spectral Flatness Mean": df['spectral_flatness'].mean(),
        "Frequency Range": (df['dominant_freq'].min(), df['dominant_freq'].max())  # Added Frequency Range
    }
    print("Sound Profile:")
    for k, v in profile.items():
        print(f"{k}: {v}")
    return profile

sound_profile = extract_sound_profile(df_filtered)

Sound Profile:
Mean Frequency: 20.78266888050502
Median Frequency: 21.533203125
Mode Frequency: 32.2998046875
Amplitude Range: (np.float64(0.0007588175), np.float64(3.5222712))
Spectral Centroid Mean: 1702.8095940567014
Spectral Bandwidth Mean: 2189.4545140190216
RMS Energy Mean: 0.005752916450344632
Zero-Crossing Rate Mean: 0.039368931934818566
Spectral Flatness Mean: 0.0006170947633412583
Frequency Range: (np.float64(0.0), np.float64(161.4990234375))


In [3]:
NUM_HOPS = 25  # Number of frequency hops
HOP_DURATION = DURATION / NUM_HOPS

def generate_complex_masking_signal_expanded(center_freq, duration, sr, amplitude_range, natural_signal):
    t = np.linspace(0, duration, int(sr * duration))
    min_freq = df_filtered['dominant_freq'].min()
    max_freq = df_filtered['dominant_freq'].max()
    
    # Generate frequency values using the min and max frequencies from the dataset
    freq_ch = np.random.uniform(min_freq, max_freq, size=len(t))
    
    # Apply frequency modulation for more energy
    fm_signal = np.sin(2 * np.pi * np.cumsum(freq_ch / sr))

    # Time-varying amplitude (Envelope)
    envelope = np.sin(2 * np.pi * 0.1 * t) ** 2  # Square the sine wave for smooth modulation
    am_signal = fm_signal * envelope

    # Expanded frequency range: Adding harmonic content
    harmonic_signal = np.sin(2 * np.pi * (center_freq * 2) * t) * 0.3  # Adding harmonic frequency
    am_signal += harmonic_signal

    # Temporal Variations
    burst_intervals = np.random.uniform(0.2, 0.5, size=5)
    burst_durations = np.random.uniform(0.2, 0.3, size=5)
    signal_with_bursts = np.zeros(len(t))

    start_idx = 0
    for interval, duration in zip(burst_intervals, burst_durations):
        end_idx = int(start_idx + duration * sr)
        signal_with_bursts[start_idx:end_idx] = am_signal[start_idx:end_idx]
        start_idx = int(end_idx + interval * sr)

    # White Noise for complexity
    white_noise = np.random.normal(0, 0.3, len(t))

    # Combine Signals
    combined_signal = signal_with_bursts + white_noise
    combined_signal = (combined_signal / np.max(np.abs(combined_signal))) * amplitude_range[1]

    return t, combined_signal

def generate_complex_masking_signal(center_freq, duration, sr, amplitude_range, natural_signal):
    t = np.linspace(0, duration, int(sr * duration))
    min_freq = df_filtered['dominant_freq'].min()
    max_freq = df_filtered['dominant_freq'].max()
    
    # Generate frequency values using the min and max frequencies from the dataset
    freq_ch = np.random.uniform(min_freq, max_freq, size=len(t))
    
    # Random frequency changes
    fm_signal = np.sin(2 * np.pi * np.cumsum(freq_ch / sr))

    num_bands = 5  # Adjust to match effective frequencies for masking
    freq_bands = [(min_freq, np.random.uniform(min_freq, max_freq)) for _ in range(num_bands)]
    
    # Amplitude Modulation: increase the depth
    amplitude_modulation = np.random.uniform(amplitude_range[0], amplitude_range[1], size=len(t)) *5  # Increased modulation depth
    am_signal = fm_signal * amplitude_modulation

    dominant_freqs = [sound_profile['Mean Frequency'], sound_profile['Median Frequency'], sound_profile['Mode Frequency']]
    for dominant_freq in dominant_freqs:
        freq_range = [dominant_freq - 10, dominant_freq + 10]  # Narrow range around dominant frequency
        band_signal = np.sin(2 * np.pi * np.random.uniform(*freq_range) * t)
        am_signal += band_signal * 0.5 

    # Temporal Variations
    burst_intervals = np.random.uniform(0.2, 0.5, size=5)  # Adjust burst intervals for better smoothing
    burst_durations = np.random.uniform(0.2, 0.3, size=5)  # Adjust burst durations for less harsh transitions
    signal_with_bursts = np.zeros(len(t))

    start_idx = 0
    for interval, duration in zip(burst_intervals, burst_durations):
        end_idx = int(start_idx + duration * sr)
        signal_with_bursts[start_idx:end_idx] = am_signal[start_idx:end_idx]
        start_idx = int(end_idx + interval * sr)

    # Scale bursts to increase power
    signal_with_bursts = signal_with_bursts * 5  # Increased burst signal power

    # Lower White Noise Level: slightly increase noise level
    white_noise = np.random.normal(0, 4, len(t))  # Increased white noise slightly

    # Frequency Hopping: amplify the signal
    freq_hopping_signal = np.zeros_like(t)
    for i in range(NUM_HOPS):
        low, high = freq_bands[np.random.randint(0, len(freq_bands))]
        freq = np.random.uniform(low, high)
        hop_start = int(i * HOP_DURATION * sr)
        hop_end = int((i + 1) * HOP_DURATION * sr)
        hop_t = t[hop_start:hop_end]
        sine_wave = np.sin(2 * np.pi * freq * hop_t)
        freq_hopping_signal[hop_start:hop_end] = sine_wave

    # Amplify the frequency hopping signal
    freq_hopping_signal = freq_hopping_signal * 3  # Amplified hopping signal

    # Combine Signals
    freq_hopping_signal = freq_hopping_signal / np.max(np.abs(freq_hopping_signal))  # Normalize
    signal_with_bursts = signal_with_bursts / np.max(np.abs(signal_with_bursts))  # Normalize
    composite_signal = freq_hopping_signal + signal_with_bursts + white_noise
    composite_signal = composite_signal / np.max(np.abs(composite_signal))  # Normalize to prevent clipping
    composite_signal = (composite_signal / np.max(np.abs(composite_signal))) * amplitude_range[1]  # Rescale to desired range

    return t, composite_signal

# Step 4: Combine Masking Signal with Natural Sound
# Load Natural Signal
wav_files = [f for f in os.listdir(AUDIO_FOLDER) if f.endswith('.wav') and INSECT_NAME in f]

if wav_files:
    random_wav_file = wav_files[0]
    natural_signal, _ = librosa.load(os.path.join(AUDIO_FOLDER, random_wav_file), sr=SR)
    # Generate Masking Signals for Key Frequencies
    masking_signals = {}
    for freq in [sound_profile['Mean Frequency'], sound_profile['Median Frequency'], sound_profile['Mode Frequency']]:
        t, signal = generate_complex_masking_signal(freq, DURATION, SR, sound_profile['Amplitude Range'], natural_signal)
        masking_signals[freq] = signal
        min_length = min(len(natural_signal), len(next(iter(masking_signals.values()))))

    natural_signal = natural_signal[:min_length]
    combined_signals = {}

    for freq, masking_signal in masking_signals.items():
        masking_signal = masking_signal[:min_length]
        combined_signal = masking_signal + natural_signal
        combined_signal = combined_signal / np.max(np.abs(combined_signal))
        combined_signals[freq] = combined_signal






# Step 5: Evaluate Masking Effectiveness
# Metrics: SNR, SMR, SOI, PME
# evaluate_signals(natural_signal, combined_signals)

In [4]:

# Optional: Save Results
# Uncomment the following lines to save the masking signals and combined signals
# import soundfile as sf
# for freq, combined_signal in combined_signals.items():
#     sf.write(f'masked_signal_{freq:.2f}Hz.wav', combined_signal, SR)


In [13]:
import numpy as np
from scipy.fft import fft

def calculate_snr(natural_signal, combined_signal):
    """Calculate Signal-to-Noise Ratio (SNR)."""
    noise = combined_signal - natural_signal
    snr = 10 * np.log10(np.sum(natural_signal ** 2) / np.sum(noise ** 2))
    return snr

def calculate_smr(natural_signal, masking_signal):
    """Calculate Signal-to-Mask Ratio (SMR)."""
    smr = 10 * np.log10(np.sum(natural_signal ** 2) / np.sum(masking_signal ** 2))
    return smr

def calculate_soi(natural_signal, masking_signal, sr):
    """Calculate Spectral Overlap Index (SOI)."""
    # Compute FFT
    natural_spectrum = np.abs(fft(natural_signal))[:len(natural_signal) // 2]
    masking_spectrum = np.abs(fft(masking_signal))[:len(masking_signal) // 2]
    overlap = np.sum(natural_spectrum * masking_spectrum)
    total_natural_energy = np.sum(natural_spectrum ** 2)
    soi = overlap / total_natural_energy
    return soi

def calculate_pme(natural_signal, combined_signal):
    """Calculate Percentage of Masking Effectiveness (PME)."""
    pme = (1 - (np.linalg.norm(natural_signal) / np.linalg.norm(combined_signal))) * 100
    return pme

# Evaluate Metrics
evaluation_results = {}
for freq, combined_signal in combined_signals.items():
    masking_signal = combined_signal - natural_signal
    snr = calculate_snr(natural_signal, combined_signal)
    smr = calculate_smr(natural_signal, masking_signal)
    soi = calculate_soi(natural_signal, masking_signal, SR)
    pme = calculate_pme(natural_signal, combined_signal)
    evaluation_results[freq] = {"SNR (dB)": snr, "SMR (dB)": smr, "SOI": soi, "PME (%)": pme}

# Print Evaluation Results
print("Masking Effectiveness Results:")
for freq, metrics in evaluation_results.items():
    print(f"Frequency {freq:.2f} Hz:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.2f}")


Masking Effectiveness Results:
Frequency 20.78 Hz:
  SNR (dB): -41.16
  SMR (dB): -41.16
  SOI: 27.66
  PME (%): 99.13
Frequency 21.53 Hz:
  SNR (dB): -40.57
  SMR (dB): -40.57
  SOI: 25.34
  PME (%): 99.06
Frequency 32.30 Hz:
  SNR (dB): -41.20
  SMR (dB): -41.20
  SOI: 28.36
  PME (%): 99.13


In [14]:
import pandas as pd

In [7]:
file_path = '/home/aditya_tilak/dataset/Aphidoletes_aphidimyza_features.csv'

In [8]:
df = pd.read_csv(file_path)
df.head()

,file_path,insect_name,spectral_centroid,spectral_bandwidth,spectral_flatness,spectral_contrast,rms_energy,zero_crossing_rate,temporal_variance,chroma,...,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,amplitude,dominant_freq
0,/mnt/external_disk/InsectSound1000/202253-16-3...,Aphidoletes_aphidimyza,1465.027287,2038.482634,0.000476,22.976416,0.001919,0.040035,0.000004,0.324713,...,9.561523,-8.403767,12.182094,-7.938270,9.289650,-4.306371,3.873267,0.483687,0.001526,129.199219
1,/mnt/external_disk/InsectSound1000/2022324-13-...,Aphidoletes_aphidimyza,1623.745005,2212.450570,0.000335,22.581436,0.002016,0.027036,0.000004,0.738201,...,16.985676,-1.534712,17.027657,-2.907636,13.624322,0.002387,9.307720,5.754075,0.001677,32.299805
2,/mnt/external_disk/InsectSound1000/2022323-13-...,Aphidoletes_aphidimyza,1834.084850,2280.284814,0.000500,23.112044,0.001600,0.038181,0.000003,0.590532,...,15.994279,-4.570681,15.827507,-5.459588,14.052932,-1.064348,9.434423,3.486636,0.001304,10.766602
3,/mnt/external_disk/InsectSound1000/2022323-13-...,Aphidoletes_aphidimyza,855.016209,1768.903067,0.000086,23.673323,0.007209,0.006081,0.000054,0.730884,...,14.331022,0.130760,13.257979,0.513568,11.323356,3.714563,8.719275,7.741486,0.006278,21.533203
4,/mnt/external_disk/InsectSound1000/2022324-13-...,Aphidoletes_aphidimyza,1563.388302,2134.204355,0.000410,23.276570,0.001850,0.034654,0.000004,0.396634,...,12.706335,-7.365408,13.594191,-7.151708,12.162776,-2.238792,7.667647,3.021446,0.001507,21.533203


In [9]:
df.drop('frequency',axis=1)

KeyError: "['frequency'] not found in axis"

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

df_features = df.drop(['file_path','insect_name'], axis = 1)

# Assuming df is your dataframe loaded with the relevant features
correlation_matrix = df_features.corr()

# Plotting the correlation matrix
plt.figure(figsize=(36, 20))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Matrix")
plt.show()


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Extract features (X) and target (y)
X = df_features.drop(columns=["dominant_freq"])  # Remove non-feature columns
y = df_features["dominant_freq"]

# Standardize the features (important for PCA)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=20)  # Keep the top 10 principal components
X_pca = pca.fit_transform(X_scaled)

# Explained variance ratio to understand how much variance each component explains
explained_variance = pca.explained_variance_ratio_
print(f'Explained Variance by each component: {explained_variance}')

# Optionally, you can plot the cumulative explained variance to decide how many components to keep
import matplotlib.pyplot as plt
plt.plot(range(1, len(explained_variance) + 1), explained_variance.cumsum(), marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA - Explained Variance')
plt.show()


In [ ]:
import numpy as np
import pandas as pd

# Get the loadings of the first 5 components
pca_loadings = pca.components_[:10]

# Create a DataFrame to view feature contributions for each component
feature_names = X.columns  # Assuming X_scaled has the feature names
pca_loadings_df = pd.DataFrame(pca_loadings.T, columns=[f'PC{i+1}' for i in range(10)], index=feature_names)

# Get the top 5 features based on the absolute value of the loadings
top_5_features = pca_loadings_df.abs().mean(axis=1).sort_values(ascending=False).head(10)
print("Top 5 Features based on PCA component loadings:")
print(top_5_features)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
print('ok')
rf.fit(X_train, y_train)
print('n ok')

# Get feature importance
importances = rf.feature_importances_

# Sort features by importance
sorted_idx = importances.argsort()[::-1]
features = X.columns

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.barh(features[sorted_idx], importances[sorted_idx])
plt.xlabel('Feature Importance')
plt.title('Feature Importance - Random Forest')
plt.show()
